# STEM Cell Population Wright-Fisher Algorithm Sensitivy Analysis for Fixation

In this example notebook we do a sensitivity analysis of the parameters of the STEM cell model using the Wright-Fisher framework and how it impact the probability of fixation of mutants as well as average fixation times.

In [1]:
# Load necessary libraries
import os
import numpy as np
import pandas as pd
from scipy.stats import gamma
import cmmlinflam as ci
import matplotlib
import plotly.graph_objects as go
from matplotlib import pyplot as plt

## Plot output of Wright-Fisher for the different species of cells

In [2]:
from plotly.subplots import make_subplots

colours = ['blue', 'red', 'green', 'purple', 'orange', 'black', 'gray', 'pink']
species = ['WT', 'A', 'B']

## Compare mean time to fixation and probability of fixation of A cells when no B are present for different population size

## Neutrality - no selectional advatange + No mutation

### Start from 1 A vs all other WT

In [3]:
# Select number of simulations
num_simulations = 10000

# Choose different population sizes
choices_N = np.arange(100, 1001, 20, dtype=np.int)

mean_computation_time = np.empty((choices_N).shape[0])
prob_fix = np.empty((choices_N).shape[0])
mean_comp_time_fix_A = np.empty((choices_N).shape[0])

for _, N in enumerate(choices_N):
    # Set initial population state WT - A - B
    initial_population = [int(N-1), 1, 0]

    # Set baseline growth rate
    alpha = 0.5

    # Set selectional advantages for mutated cells
    s = 0
    r = 0

    # Set mutation rates
    mu_A = 0
    mu_B = 0

    # Coalesce into paramater vector
    parameters = initial_population
    parameters.extend([alpha, s, r, mu_A, mu_B])

    # Instantiate algorithm
    algorithm = ci.StemWF()

    computation_time = np.empty(num_simulations, dtype=np.int)
    fixed_state = np.empty(num_simulations, dtype=np.str)

    for sim in range(num_simulations):
        computation_time[sim], fixed_state[sim] = algorithm.simulate_fixation(parameters)

    mean_computation_time[_] = np.mean(computation_time)
    prob_fix[_] = (fixed_state == 'A').sum()/num_simulations

    if len(computation_time[(fixed_state == 'A')]) == 0:
        mean_comp_time_fix_A[_] = 0
    else:
        mean_comp_time_fix_A[_] = np.mean(computation_time[(fixed_state == 'A')])


### Plot mean computation times to fixation

In [4]:
# Trace names - represent the transition probabilities used for the simulation
fig = go.Figure()

# Add traces of the transition probabilities
fig.add_trace(
    go.Scatter(
        y = mean_computation_time,
        x = choices_N,
        mode = 'lines',
        line_color = colours[2]
    )
)

fig.update_layout(
    title='Mean computation times to fixation for edge case with no B cells - No selectional advantage + No mutation',
    width=1000, 
    height=600,
    plot_bgcolor='white',
    xaxis=dict(linecolor='black'),
    yaxis=dict(linecolor='black'),
    )

### Plot mean computation times to fixation of A

In [5]:
# Trace names - represent the transition probabilities used for the simulation
fig = go.Figure()

# Add traces of the transition probabilities
fig.add_trace(
    go.Scatter(
        y = mean_comp_time_fix_A,
        x = choices_N,
        mode = 'lines',
        line_color = colours[2]
    )
)

fig.update_layout(
    title='Mean computation times to fixation of A for edge case with no B cells - No selectional advantage + No mutation',
    width=1000, 
    height=600,
    plot_bgcolor='white',
    xaxis=dict(linecolor='black'),
    yaxis=dict(linecolor='black'),
    )

### Plot probability to fixation of A

In [6]:
# Trace names - represent the transition probabilities used for the simulation
fig = go.Figure()

# Add traces of the transition probabilities
fig.add_trace(
    go.Scatter(
        y = prob_fix,
        x = choices_N,
        mode = 'lines',
        line_color = colours[2]
    )
)

fig.update_layout(
    title='Probability to fixation of A for edge case with no B cells - No selectional advantage + No mutation',
    width=1000, 
    height=600,
    plot_bgcolor='white',
    xaxis=dict(linecolor='black'),
    yaxis=dict(linecolor='black'),
    )

## Compare mean time to fixation and probability of fixation of A cells when no B are present for different population size

## Selectional advatange + No mutation

### Start from 1 A vs all other WT

In [7]:
# Select number of simulations
num_simulations = 10000

# Choose different population sizes
small_N = [100, 1000]

# Choose different selective advantage
choices_s = np.arange(0, 0.55, 0.05)

mean_computation_time = np.empty((choices_s.shape[0], 2))
prob_fix = np.empty((choices_s.shape[0], 2))
mean_comp_time_fix_A = np.empty((choices_s.shape[0], 2))
comput_time_fix_A = [[],[]]

for _, s in enumerate(choices_s):
    for n, N in enumerate(small_N):
        # Set initial population state WT - A - B
        initial_population = [int(N-1), 1, 0]

        # Set baseline growth rate
        alpha = 0.5

        # Set selectional advantages for mutated cells
        s = 0
        r = 0

        # Set mutation rates
        mu_A = 0
        mu_B = 0

        # Coalesce into paramater vector
        parameters = initial_population
        parameters.extend([alpha, s, r, mu_A, mu_B])

        # Instantiate algorithm
        algorithm = ci.StemWF()

        computation_time = np.empty((num_simulations, choices_s.shape[0], len(small_N)), dtype=np.int)
        fixed_state = np.empty(num_simulations, dtype=np.str)

        for sim in range(num_simulations):
            computation_time[sim, _, n], fixed_state[sim] = algorithm.simulate_fixation(parameters)

        mean_computation_time[_, n] = np.mean(computation_time[:, _, n])
        prob_fix[_, n] = (fixed_state == 'A').sum()/num_simulations

        if len(computation_time[:, _, n][(fixed_state == 'A')]) == 0:
            mean_comp_time_fix_A[_, n] = 0
            comput_time_fix_A[n].append([])
        else:
            mean_comp_time_fix_A[_, n] = np.mean(computation_time[:, _, n][(fixed_state == 'A')])
            comput_time_fix_A[n].append(computation_time[:, _, n][(fixed_state == 'A')].tolist())

### Plot mean computation times to fixation

In [8]:
# Trace names - represent the transition probabilities used for the simulation
trace_name = ['Population size = {}'.format(N) for N in small_N]

fig = go.Figure()

# Add traces of the transition probabilities
for n, _ in enumerate(small_N):
    fig.add_trace(
        go.Scatter(
            y = mean_computation_time[:, n],
            x = choices_s,
            mode = 'lines',
            name = trace_name[n],
            line_color = colours[n]
        )
    )

fig.update_layout(
    title='Mean computation times to fixation for edge case with no B cells - Selectional advantage + No mutation',
    width=1000, 
    height=600,
    plot_bgcolor='white',
    xaxis=dict(linecolor='black'),
    yaxis=dict(linecolor='black'),
    )

### Plot mean computation times to fixation of A

In [9]:
fig = go.Figure()

# Add traces of the transition probabilities
for n, _ in enumerate(small_N):
    fig.add_trace(
        go.Scatter(
            y = mean_comp_time_fix_A[:, n],
            x = choices_s,
            mode = 'lines',
            name = trace_name[n],
            line_color = colours[n]
        )
    )

fig.update_layout(
    title='Mean computation times to fixation at A for edge case with no B cells - Selectional advantage + No mutation',
    width=1000, 
    height=600,
    plot_bgcolor='white',
    xaxis=dict(linecolor='black'),
    yaxis=dict(linecolor='black'),
    )

### Plot probability of fixation of A

In [10]:
fig = go.Figure()

# Add traces of the transition probabilities
for n, _ in enumerate(small_N):
    fig.add_trace(
        go.Scatter(
            y = prob_fix[:, n],
            x = choices_s,
            mode = 'lines',
            name = trace_name[n],
            line_color = colours[n]
        )
    )

fig.update_layout(
    title='Probability of fixation at A for edge case with no B cells - Selectional advantage + No mutation',
    width=1000, 
    height=600,
    plot_bgcolor='white',
    xaxis=dict(linecolor='black'),
    yaxis=dict(linecolor='black'),
    )

### Boxplot of times to fixation for different population sizes and selective advatantages

In [11]:
# Trace names - represent the transition probabilities used for the simulation
trace_name = ['Population size = {}'.format(N) for N in small_N]

fig = go.Figure()

# Add traces of the transition probabilities
for n, _ in enumerate(small_N):
    fig.add_trace(
        go.Box(
            mean=np.mean(computation_time[:, :, n], axis=0),
            median=np.quantile(computation_time[:, :, n], 0.5, axis=0),
            q1=np.quantile(computation_time[:, :, n], 0.025, axis=0),
            q3=np.quantile(computation_time[:, :, n], 0.975, axis=0),
            name = trace_name[n],
            marker_color = colours[n]
        )
    )

fig.update_layout(
    boxmode='group',
    title='Mean computation times to fixation for edge case with no B cells - Selectional advantage + No mutation',
    width=1000, 
    height=600,
    plot_bgcolor='white',
    xaxis=dict(linecolor='black'),
    yaxis=dict(linecolor='black'),
)

### Boxplot of times to fixation to A for different population sizes and selective advatantages

In [12]:
# Trace names - represent the transition probabilities used for the simulation
trace_name = ['Population size = {}'.format(N) for N in small_N]

fig = go.Figure()

# Add traces of the transition probabilities
for n, _ in enumerate(small_N):
    fig.add_trace(
        go.Box(
            mean=[np.mean(np.asarray(j)) for j in comput_time_fix_A[n]],
            median=[np.quantile(np.asarray(j), 0.5) for j in comput_time_fix_A[n]],
            q1=[np.quantile(np.asarray(j), 0.025) for j in comput_time_fix_A[n]],
            q3=[np.quantile(np.asarray(j), 0.975) for j in comput_time_fix_A[n]],
            name = trace_name[n],
            marker_color = colours[n]
        )
    )

fig.update_layout(
    boxmode='group',
    title='Mean computation times to fixation to A for edge case with no B cells - Selectional advantage + No mutation',
    width=1000, 
    height=600,
    plot_bgcolor='white',
    xaxis=dict(linecolor='black'),
    yaxis=dict(linecolor='black'),
)